In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

## Loading MNIST Dataset

In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
mnist.train.num_examples

55000

In [4]:
mnist.test.num_examples

10000

In [5]:
mnist.train.labels.shape

(55000, 10)

In [6]:
mnist.test.labels.shape

(10000, 10)

## Initializing Weights

In [7]:
def initialize_weights(filter_shape):
    init_random_dist = tf.truncated_normal(filter_shape,stddev=0.1)
    return (tf.Variable(init_random_dist))

## Initializing Bias

In [8]:
def initialize_bias(bias_shape):
    initial_bias_vals = tf.constant(0.1,shape=bias_shape)
    return (tf.Variable(initial_bias_vals))

## Creating convolution layer

In [9]:
def create_convolution_layer(inputs, filter_shape):
    filter_initialized_with_weights = initialize_weights(filter_shape)
    conv_layer_outputs = tf.nn.conv2d(inputs,filter_initialized_with_weights, strides=[1,1,1,1],padding ='SAME')
    return (conv_layer_outputs)

## Creating Relu Layer

In [10]:
def create_relu_layer(inputs, filter_shape):
    b = initialize_bias([filter_shape[3]])
    relu_layer_outputs = tf.nn.relu(inputs+b)
    return (relu_layer_outputs)

## Creating Pooling Layer

In [11]:
def create_max_pool(inputs):
    pooling_layer_outputs = tf.nn.max_pool(inputs, ksize=[1,2,2,1],strides = [1,2,2,1],padding='SAME')
    return (pooling_layer_outputs)

## Creating Fully Connected Layer

In [12]:
def create_fully_connected_layer(inputs, output_size):
    input_size = int(inputs.get_shape()[1])
    W = initialize_weights([input_size, output_size])
    b = initialize_bias([output_size])
    fcbias_outputs=tf.matmul(inputs,W)+b
    return (fcbias_outputs)

In [13]:
x = tf.placeholder(tf.float32,shape=[None, 784])

In [14]:
y_true = tf.placeholder(tf.float32,shape=[None, 10])

In [15]:
x_image = tf.reshape(x,[-1,28,28,1])

In [16]:
conv_layer1=create_convolution_layer(x_image,filter_shape=[5,5,1,32])

In [17]:
conv_relu_layer1=create_relu_layer(conv_layer1,filter_shape=[5,5,1,32])

In [18]:
pooling_layer1 = create_max_pool(conv_relu_layer1)

## Flattening pooling layer output to 1-dimension

In [19]:
poolig_layer_flatten = tf.reshape(pooling_layer1, [-1,14*14*32])

In [20]:
fc_layer_1_outputs = create_fully_connected_layer(poolig_layer_flatten, output_size=1024)

In [21]:
fc_relu_layer_1_outputs = tf.nn.relu(fc_layer_1_outputs)

In [22]:
dropout_hold_prob = tf.placeholder(tf.float32)
dropout_outputs = tf.nn.dropout(fc_relu_layer_1_outputs, keep_prob=dropout_hold_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [23]:
y_pred = create_fully_connected_layer(dropout_outputs,output_size=10)

## Loss Function

In [24]:
softmax_cross_entropy_loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true,logits=y_pred)
cross_entropy_mean = tf.reduce_mean(softmax_cross_entropy_loss)

## Optimizing model will update weights and bias of model

In [25]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)

In [26]:
cnn_trainer=optimizer.minimize(cross_entropy_mean)

In [27]:
vars_initializer = tf.global_variables_initializer()
steps=5000

In [28]:
with tf.Session() as sess:
    sess.run(vars_initializer)
    results=[]
    for i in range(steps):
        batch_x,batch_y = mnist.train.next_batch(50)
        sess.run(cnn_trainer,feed_dict = {x:batch_x, y_true:batch_y,dropout_hold_prob:0.5})
        if i%100==0:
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
            acc = tf.reduce_mean(tf.cast(matches,tf.float32))
            test_accuracy = sess.run(acc, feed_dict = {x:mnist.test.images,y_true:mnist.test.labels,dropout_hold_prob:1.0})
            results.append(test_accuracy)
            print(f"ON STEP:{i} Accuracy:{round((test_accuracy*100),2)}")
            print("\n")


User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_DUPLICATE_LIB_OK=True
   KMP_INIT_AT_FORK=FALSE
   KMP_SETTINGS=1
   OMP_NUM_THREADS=8

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_HAND_THREAD=false
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_FORKJOIN_FRAMES=true
   KMP_FORKJOIN_FRAMES_MODE=3
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_ITT_PREPARE_DELAY=0
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_IN

ON STEP:0 Accuracy:12.9


ON STEP:100 Accuracy:90.34


ON STEP:200 Accuracy:93.63


ON STEP:300 Accuracy:95.42


ON STEP:400 Accuracy:96.02


ON STEP:500 Accuracy:96.37


ON STEP:600 Accuracy:97.1


ON STEP:700 Accuracy:97.19


ON STEP:800 Accuracy:97.42


ON STEP:900 Accuracy:97.82


ON STEP:1000 Accuracy:97.87


ON STEP:1100 Accuracy:98.07


ON STEP:1200 Accuracy:97.92


ON STEP:1300 Accuracy:98.16


ON STEP:1400 Accuracy:98.08


ON STEP:1500 Accuracy:98.28


ON STEP:1600 Accuracy:98.26


ON STEP:1700 Accuracy:98.26


ON STEP:1800 Accuracy:98.61


ON STEP:1900 Accuracy:98.37


ON STEP:2000 Accuracy:98.29


ON STEP:2100 Accuracy:98.76


ON STEP:2200 Accuracy:98.24


ON STEP:2300 Accuracy:98.59


ON STEP:2400 Accuracy:98.57


ON STEP:2500 Accuracy:98.56


ON STEP:2600 Accuracy:98.72


ON STEP:2700 Accuracy:98.41


ON STEP:2800 Accuracy:98.56


ON STEP:2900 Accuracy:98.63


ON STEP:3000 Accuracy:98.58


ON STEP:3100 Accuracy:98.66


ON STEP:3200 Accuracy:98.72


ON STEP:3300 Accuracy:98

In [29]:
round(np.mean(results)*100,2)

96.34